In [ ]:
%matplotlib inline
import torch
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score

ROOT_PATH = "C:\\Users\\info\\Documents\\Polito\\MSc\\ML-AI\\HW3\\data\\"

In [21]:


# function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def plot_kernel(model):
    model_weights = model.state_dict()
    fig = plt.figure()
    plt.figure(figsize=(10,10))
    for idx, filt  in enumerate(model_weights['conv1.weight']):
        if idx >= 32:
            continue
        plt.subplot(4,8, idx + 1)
        plt.imshow(filt[0, :, :], cmap="gray")
        plt.axis('off')
    plt.show()

def plot_kernel_output(model,images):
    fig1 = plt.figure()
    plt.figure(figsize=(1,1))
    img_normalized = (images[0] - images[0].min()) / (images[0].max() - images[0].min())
    plt.imshow(img_normalized.numpy().transpose(1,2,0))
    plt.show()
    output = model.conv1(images)
    layer_1 = output[0, :, :, :]
    layer_1 = layer_1.data
    fig = plt.figure()
    plt.figure(figsize=(10,10))
    for idx, filt  in enumerate(layer_1):
        if idx >= 32:
            continue
        plt.subplot(4,8, idx + 1)
        plt.imshow(filt, cmap="gray")
        plt.axis('off')
    plt.show()

def test_accuracy(net, dataloader):  
    # check accuracy on whole test set
    correct = 0
    total = 0
    net.eval() # important for deactivating dropout and correctly use batchnorm accumulated statistics
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print("Accuracy of the network on the test set: {}".format(accuracy))
    return accuracy
   
n_classes = 100 
      
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #conv2d first parameter is the number of kernels at input (you get it from the output value of the previous layer)
        #conv2d second parameter is the number of kernels you wanna have in your convolution, so it will be the n. of kernels at output.
        #conv2d third, fourth and fifth parameters are, as you can read, kernel_size, stride and zero padding :)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv_final = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.fc1 = nn.Linear(64 * 4 * 4, 4096)
        self.fc2 = nn.Linear(4096, n_classes) #last FC for classification 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.pool(self.conv_final(x)))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        # hint: dropout goes here
        x = self.fc2(x)
        return x

In [ ]:
# transform are heavily used to do simple and complex transformation and data augmentation
transform_train = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=0,drop_last=True)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=0,drop_last=True)
dataiter = iter(trainloader)

net = CNN()
net = net.cuda()
criterion = nn.CrossEntropyLoss().cuda() # it already does softmax computation for use!
optimizer = optim.Adam(net.parameters(), lr=0.0001) # better convergency w.r.t simple SGD :)
print('Data ready')

In [ ]:
n_loss_print = len(trainloader)  # print every epoch, use smaller numbers if you wanna print loss more often!

n_epochs = 20
for epoch in range(n_epochs):  # loop over the dataset multiple times
    net.train() # important for activating dropout and correctly train batchnorm
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs and cast them into cuda wrapper
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % n_loss_print == (n_loss_print -1):    
            print("[{}, {}] loss: {}".format(epoch + 1, i + 1, round(running_loss / n_loss_print, 3)))
            running_loss = 0.0
    test_accuracy(net,testloader)
print('Finished Training')

In [ ]:
n_loss_print = len(trainloader)  # print every epoch, use smaller numbers if you wanna print loss more often!
n_epochs = 20
stats = np.empty((n_epochs,2))
for epoch in range(n_epochs):  # loop over the dataset multiple times
    net.train() # important for activating dropout and correctly train batchnorm
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs and cast them into cuda wrapper
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % n_loss_print == (n_loss_print -1):
            print("[{}, {}] loss: {}".format(epoch + 1, i + 1, round(running_loss / n_loss_print, 3)))
            stats[epoch][1] = running_loss / n_loss_print
            running_loss = 0.0
    stats[epoch][0] = test_accuracy(net,testloader)
print('Finished Training')
param_range = np.arange(n_epochs+1)
plt.ylim(0.0, 100.0)
plt.semilogx(param_range, stats[:,0], label='Accuracy', lw=lw)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()


In [ ]:
stats = np.empty(n_epochs)
print(stats.shape)
stats = np.empty((n_epochs,2))
param_range = np.arange(n_epochs+1)
lw = 1
plt.ylim(0.0, 100.0)
plt.semilogx(param_range, stats[:,0], label='Accuracy', lw=lw)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()